# FullStack GPT: #6.0 ~ #6.10

- [x] Implement a complete RAG pipeline with a Stuff Documents chain.
- [x] You must implement the chain manually.
- [x] Give a ConversationBufferMemory to the chain.
- [x] Use this document to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- [x] Ask the following questions to the chain:
    - Is Aaronson guilty?
    - What message did he write in the table?
    - Who is Julia?

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})

def get_history(_):
    return memory.load_memory_variables({})["history"]

loader = TextLoader("../files/document.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=50,
)
docs = loader.load_and_split(text_splitter=splitter)

cache_dir = LocalFileStore("../.cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages=True)
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
prompt = ChatPromptTemplate.from_messages([
    ("system", """
     You are a helpful assistant. Answer the question based solely on the given context and chat history. If you don't know the answer, don't make it up.
     -----
     {context}
    """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = {
    "context": retriever, 
    "history": RunnableLambda(get_history), 
    "question": RunnablePassthrough()
    } | prompt | llm

Created a chunk of size 538, which is longer than the specified 500
Created a chunk of size 511, which is longer than the specified 500


In [ ]:
from langchain.callbacks import get_openai_callback
questions = ["Is Aaronson guilty?", "What message did he write in the table?", "Who is Julia?"]

for question in questions:
    print(question)
    chain.invoke(question)
    print("\n")

Is Aaronson guilty?
In the context provided, Winston reflects that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and concludes that it had never existed. This suggests that the Party's claims about Aaronson's guilt are questionable and likely a manipulation of truth. Therefore, while the Party asserts that Aaronson is guilty, the reality of his guilt is uncertain.

What message did he write in the table?
Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE" on the table. Later, he also wrote "GOD IS POWER."

Who is Julia?
Julia is a character in the story who is Winston's love interest. She works for the Party but secretly opposes its oppressive regime. Julia engages in a romantic relationship with Winston, and their connection symbolizes rebellion against the Party's control over personal relationships and individual freedoms. Throu